In [1]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from work.utils.dataset import PandasDataset, RGB2HSVTransform
from work.utils.models import EfficientNet
from work.utils.metrics import evaluation
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import albumentations

In [2]:
output_dimensions = 5
data_dir = '../../dataset'
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv(f"../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: '/home/woshington/Projects/Doutorado/work/efficientnet-b0-08094119.pth'
}

In [3]:
print("Cuda", device)

Cuda cuda


In [4]:
val_transform =albumentations.Compose([
    RGB2HSVTransform()
])

In [5]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=val_transform),
    batch_size=2,
    shuffle=False,
)

In [6]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device)
model.load_state_dict(
    torch.load(
        "models/with-noise-hsv.pth",
        weights_only=True
    )
)

response_0 = evaluation(model, dataloader, device)
print("without treatment", response_0[0])
cm_0 = confusion_matrix(response_0[1][1], response_0[1][0])

Loaded pretrained weights for efficientnet-b0


100%|██████████| 796/796 [10:46<00:00,  1.23it/s]


without treatment {'val_acc': {'mean': 60.14032660722732, 'std': 1.2192787266294804, 'ci_5': 58.103013038635254, 'ci_95': 62.12625473737717}, 'val_kappa': {'mean': 0.8249467660882137, 'std': 0.011113349529536302, 'ci_5': 0.8060289191372576, 'ci_95': 0.8432585218647115}, 'val_f1': {'mean': 0.5467223304510117, 'std': 0.013067634771398565, 'ci_5': 0.5254088044166565, 'ci_95': 0.5686790227890015}, 'val_recall': {'mean': 0.5434830890893936, 'std': 0.013012339808087846, 'ci_5': 0.5219810962677002, 'ci_95': 0.5654451847076416}, 'val_precision': {'mean': 0.5651042701005936, 'std': 0.012928693509566835, 'ci_5': 0.5436622619628906, 'ci_95': 0.5862693935632706}}
